In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [15]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

Question 1

In [3]:
# Get data frommthe given site 
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
df = ipo_dfs[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [16]:
df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Filling Date,Avg_price,Shares_offered_value
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,NaN,2024-05-03,NaN,NaN
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,2024-04-29,3.0,11250000.0
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,2024-04-29,10.0,60000000.0
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000.0,2024-04-26,10.0,50000000.0
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,NaN,2024-04-26,NaN,NaN


In [17]:
# Convert 'Filing Date' to datetime
# df['Filing Date'] = pd.to_datetime(df['Filing Date'])

df['Filling Date'] = pd.to_datetime(df['Filing Date'])



In [18]:
# Convert 'Shares Offered' to float64
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'].replace('-', float('NaN')))


In [19]:
import re

In [20]:

# Calculate 'Avg_price'
df['Avg_price'] = df['Price Range'].apply(lambda x: np.nan if x == '-' or not x else (float(re.findall(r'\d+\.?\d*', x)[0]) if len(re.findall(r'\d+\.?\d*', x)) == 1 else (float(re.findall(r'\d+\.?\d*', x)[0]) + float(re.findall(r'\d+\.?\d*', x)[1])) / 2))

In [10]:
df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Filling Date,Avg_price
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,NaN,2024-05-03,NaN
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,2024-04-29,3.00
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,2024-04-29,10.00
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000.0,2024-04-26,10.00
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,NaN,2024-04-26,NaN
...,...,...,...,...,...,...,...
321,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,2020-01-21,9.00
322,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,2020-01-21,10.00
323,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,2019-12-09,9.00
324,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,2019-10-04,4.00


In [22]:
def calculate_avg_price(price_range):
    if price_range == '-' or not price_range:  # If no price specified or it's "-"
        return np.nan
    prices = re.findall(r'\d+\.?\d*', price_range)  # Extracting prices
    if len(prices) == 1:  # If only one price provided
        return float(prices[0])
    elif len(prices) == 2:  # If a range of prices provided
        return (float(prices[0]) + float(prices[1])) / 2
    else:  # Invalid price format
        return np.nan

df['Avg_price'] = df['Price Range'].apply(calculate_avg_price)

# Define 'Shares_offered_value'
def calculate_shares_offered_value(row):
    if pd.notnull(row['Shares Offered']) and pd.notnull(row['Avg_price']):
        return row['Shares Offered'] * row['Avg_price']
    else:
        return np.nan

df['Shares_offered_value'] = df.apply(calculate_shares_offered_value, axis=1)

df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Filling Date,Avg_price,Shares_offered_value
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,NaN,2024-05-03,NaN,NaN
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,2024-04-29,3.00,11250000.0
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,2024-04-29,10.00,60000000.0
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000.0,2024-04-26,10.00,50000000.0
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,NaN,2024-04-26,NaN,NaN
...,...,...,...,...,...,...,...,...
321,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,2020-01-21,9.00,13500000.0
322,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,2020-01-21,10.00,50000000.0
323,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,2019-12-09,9.00,22500000.0
324,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,2019-10-04,4.00,17200000.0


In [24]:
# Filter data for 2023 and Fridays
filtered_df = df[(df['Filing Date'].dt.year == 2023) & (df['Filing Date'].dt.dayofweek == 4)]

In [25]:
# Sum 'Shares_offered_value' for filtered records
total_sum_usd = filtered_df['Shares_offered_value'].sum()

In [26]:
# Convert sum to millions of USD and round to closest integer
total_sum_millions = round(total_sum_usd / 1e6)

print("Total sum in $m (millions of USD) for all filings during 2023 that happened on Fridays:", total_sum_millions)

Total sum in $m (millions of USD) for all filings during 2023 that happened on Fridays: 286


Question 2

In [ ]:
### Question 2:  IPOs "Fixed days hold" strategy

In [ ]:
# **Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

In [27]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [28]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [30]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, df], ignore_index=True)

In [37]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,Filing Date,Price Range,Shares Offered,Filling Date,Avg_price,Shares_offered_value
0,2024-05-08,NNE,Nano Nuclear Energy Inc.,$4.00,$5.19,29.75%,NaT,NaN,NaN,NaT,NaN,NaN


In [38]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   IPO Date              65 non-null     datetime64[ns]
 1   Symbol                391 non-null    object        
 2   Company Name          391 non-null    object        
 3   IPO Price             65 non-null     object        
 4   Current               65 non-null     object        
 5   Return                65 non-null     object        
 6   Filing Date           326 non-null    datetime64[ns]
 7   Price Range           326 non-null    object        
 8   Shares Offered        252 non-null    float64       
 9   Filling Date          326 non-null    datetime64[ns]
 10  Avg_price             258 non-null    float64       
 11  Shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](3), float64(3), object(6)
memory usage: 36.8+ KB


In [39]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'])

In [40]:
# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,Filing Date,Price Range,Shares Offered,Filling Date,Avg_price,Shares_offered_value


In [41]:
nvo_df = yf.download(tickers = "NVO",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [42]:
# big plus of an API's data: good typing from the box!
# notice DatetimeIndex - it is a recognised date
nvo_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10847 entries, 1981-04-30 to 2024-05-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       10847 non-null  float64
 1   High       10847 non-null  float64
 2   Low        10847 non-null  float64
 3   Close      10847 non-null  float64
 4   Adj Close  10847 non-null  float64
 5   Volume     10847 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 593.2 KB


In [43]:
# filter on date (index)
# nvo_df_filtered_from_2020 = nvo_df[nvo_df.index>='2020-01-01']
nvo_df_filtered_from_2024 = nvo_df[nvo_df.index>='2024-03-01']